In [1]:
import numpy as np 
import pandas as pd 
import tqdm
import string 

In [2]:
import seaborn as sns
import xgboost as xgb
import lightgbm as lgbm
from sklearn.svm import LinearSVC 
from sklearn.calibration import CalibratedClassifierCV 

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes

In [4]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 

import nltk # for stopwords 
from nltk.corpus import stopwords
import gensim # for Word2Vec embeddings 
from sklearn.feature_extraction.text import CountVectorizer

from konlpy.tag import Mecab, Hannanum

In [16]:
train = pd.read_csv('lgbm_train_df.csv') 
test = pd.read_csv('lgbm_test_df.csv')

In [18]:
x_train = train.drop(['n_id','info', 'title', 'content', 'title_tokenized', 'content_tokenized'], axis = 1) 
y_train = train['info'] 

x_train = np.asarray(x_train) 
y_train = np.asarray(y_train)

In [20]:
x_test = test.drop(['n_id','id','title', 'content', 'title_tokenized', 'content_tokenized'],axis=1) 

In [21]:
x_test = np.asarray(x_test)

In [22]:
x_train.shape, y_train.shape, x_test.shape

((118745, 63), (118745,), (142565, 63))

In [29]:
k = 5 
models = [] 
kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 888)
for n_fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)):  
    train_x, val_x = x_train[train_idx], x_train[val_idx] 
    train_y, val_y = y_train[train_idx], y_train[val_idx] 
    
    param = {
        'eta': 0.01, 
        'max_depth': 5, 
        'objective': 'binary:logistic', 
        'eval_metric': 'logloss', 
        'colsample_bytree': 0.9, 
        'sub_sample': 0.75
    }
    dtrain = xgb.DMatrix(data = train_x, label = train_y) 
    dval = xgb.DMatrix(data = val_x, label = val_y)   
    wlist = [(dtrain, 'train'), (dval, 'eval')]
    model = xgb.train(params = param, dtrain = dtrain, num_boost_round = 5000, evals = wlist, verbose_eval = 10, early_stopping_rounds = 100) 
    models.append(model)
    

[23:55:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { sub_sample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.68489	eval-logloss:0.68492
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 100 rounds.
[10]	train-logloss:0.61095	eval-logloss:0.61097
[20]	train-logloss:0.54912	eval-logloss:0.54917
[30]	train-logloss:0.49674	eval-logloss:0.49676
[40]	train-logloss:0.45202	eval-logloss:0.45194
[50]	train-logloss:0.41333	eval-logloss:0.41318
[60]	train-logloss:0.37978	eval-logloss:0.37954
[70]	train-logloss:0.35036	eval-logloss:0.35017
[80]	train-logloss:0.32446	eval-logloss:0.32430
[90]	train-logloss:0.30166	eval-logloss:0.30153
[100]	train

[1560]	train-logloss:0.03883	eval-logloss:0.04701
[1570]	train-logloss:0.03861	eval-logloss:0.04682
[1580]	train-logloss:0.03841	eval-logloss:0.04665
[1590]	train-logloss:0.03816	eval-logloss:0.04644
[1600]	train-logloss:0.03797	eval-logloss:0.04628
[1610]	train-logloss:0.03780	eval-logloss:0.04613
[1620]	train-logloss:0.03760	eval-logloss:0.04597
[1630]	train-logloss:0.03745	eval-logloss:0.04583
[1640]	train-logloss:0.03725	eval-logloss:0.04567
[1650]	train-logloss:0.03708	eval-logloss:0.04552
[1660]	train-logloss:0.03692	eval-logloss:0.04539
[1670]	train-logloss:0.03678	eval-logloss:0.04527
[1680]	train-logloss:0.03661	eval-logloss:0.04513
[1690]	train-logloss:0.03645	eval-logloss:0.04499
[1700]	train-logloss:0.03628	eval-logloss:0.04487
[1710]	train-logloss:0.03610	eval-logloss:0.04473
[1720]	train-logloss:0.03594	eval-logloss:0.04460
[1730]	train-logloss:0.03576	eval-logloss:0.04445
[1740]	train-logloss:0.03560	eval-logloss:0.04431
[1750]	train-logloss:0.03544	eval-logloss:0.04418


[3200]	train-logloss:0.01953	eval-logloss:0.03147
[3210]	train-logloss:0.01944	eval-logloss:0.03140
[3220]	train-logloss:0.01936	eval-logloss:0.03132
[3230]	train-logloss:0.01929	eval-logloss:0.03127
[3240]	train-logloss:0.01923	eval-logloss:0.03123
[3250]	train-logloss:0.01916	eval-logloss:0.03117
[3260]	train-logloss:0.01909	eval-logloss:0.03112
[3270]	train-logloss:0.01901	eval-logloss:0.03105
[3280]	train-logloss:0.01892	eval-logloss:0.03098
[3290]	train-logloss:0.01883	eval-logloss:0.03091
[3300]	train-logloss:0.01875	eval-logloss:0.03083
[3310]	train-logloss:0.01866	eval-logloss:0.03077
[3320]	train-logloss:0.01859	eval-logloss:0.03072
[3330]	train-logloss:0.01852	eval-logloss:0.03066
[3340]	train-logloss:0.01844	eval-logloss:0.03059
[3350]	train-logloss:0.01837	eval-logloss:0.03054
[3360]	train-logloss:0.01830	eval-logloss:0.03049
[3370]	train-logloss:0.01821	eval-logloss:0.03043
[3380]	train-logloss:0.01815	eval-logloss:0.03037
[3390]	train-logloss:0.01808	eval-logloss:0.03032


[4840]	train-logloss:0.01047	eval-logloss:0.02435
[4850]	train-logloss:0.01043	eval-logloss:0.02432
[4860]	train-logloss:0.01037	eval-logloss:0.02428
[4870]	train-logloss:0.01032	eval-logloss:0.02424
[4880]	train-logloss:0.01028	eval-logloss:0.02422
[4890]	train-logloss:0.01025	eval-logloss:0.02420
[4900]	train-logloss:0.01022	eval-logloss:0.02417
[4910]	train-logloss:0.01018	eval-logloss:0.02414
[4920]	train-logloss:0.01014	eval-logloss:0.02411
[4930]	train-logloss:0.01011	eval-logloss:0.02409
[4940]	train-logloss:0.01007	eval-logloss:0.02405
[4950]	train-logloss:0.01004	eval-logloss:0.02403
[4960]	train-logloss:0.01000	eval-logloss:0.02399
[4970]	train-logloss:0.00997	eval-logloss:0.02397
[4980]	train-logloss:0.00992	eval-logloss:0.02393
[4990]	train-logloss:0.00988	eval-logloss:0.02390
[4999]	train-logloss:0.00986	eval-logloss:0.02388
[00:03:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { sub_sample } might not be used.

  This may not be accurate du

[1390]	train-logloss:0.04269	eval-logloss:0.04879
[1400]	train-logloss:0.04248	eval-logloss:0.04861
[1410]	train-logloss:0.04229	eval-logloss:0.04846
[1420]	train-logloss:0.04213	eval-logloss:0.04831
[1430]	train-logloss:0.04192	eval-logloss:0.04813
[1440]	train-logloss:0.04172	eval-logloss:0.04795
[1450]	train-logloss:0.04151	eval-logloss:0.04778
[1460]	train-logloss:0.04135	eval-logloss:0.04763
[1470]	train-logloss:0.04115	eval-logloss:0.04745
[1480]	train-logloss:0.04096	eval-logloss:0.04729
[1490]	train-logloss:0.04078	eval-logloss:0.04713
[1500]	train-logloss:0.04057	eval-logloss:0.04696
[1510]	train-logloss:0.04036	eval-logloss:0.04679
[1520]	train-logloss:0.04013	eval-logloss:0.04660
[1530]	train-logloss:0.03996	eval-logloss:0.04647
[1540]	train-logloss:0.03976	eval-logloss:0.04630
[1550]	train-logloss:0.03960	eval-logloss:0.04616
[1560]	train-logloss:0.03943	eval-logloss:0.04602
[1570]	train-logloss:0.03925	eval-logloss:0.04588
[1580]	train-logloss:0.03911	eval-logloss:0.04575


...

[2690]	train-logloss:0.02391	eval-logloss:0.03433
[2700]	train-logloss:0.02380	eval-logloss:0.03425
[2710]	train-logloss:0.02368	eval-logloss:0.03417
[2720]	train-logloss:0.02356	eval-logloss:0.03409
[2730]	train-logloss:0.02344	eval-logloss:0.03399
[2740]	train-logloss:0.02335	eval-logloss:0.03393
[2750]	train-logloss:0.02326	eval-logloss:0.03386
[2760]	train-logloss:0.02309	eval-logloss:0.03375
[2770]	train-logloss:0.02296	eval-logloss:0.03364
[2780]	train-logloss:0.02283	eval-logloss:0.03355
[2790]	train-logloss:0.02274	eval-logloss:0.03348
[2800]	train-logloss:0.02266	eval-logloss:0.03340
[2810]	train-logloss:0.02256	eval-logloss:0.03333
[2820]	train-logloss:0.02246	eval-logloss:0.03327
[2830]	train-logloss:0.02239	eval-logloss:0.03320
[2840]	train-logloss:0.02230	eval-logloss:0.03314
[2850]	train-logloss:0.02218	eval-logloss:0.03304
[2860]	train-logloss:0.02211	eval-logloss:0.03299
[2870]	train-logloss:0.02200	eval-logloss:0.03290
[2880]	train-logloss:0.02192	eval-logloss:0.03285


[4330]	train-logloss:0.01273	eval-logloss:0.02594
[4340]	train-logloss:0.01269	eval-logloss:0.02591
[4350]	train-logloss:0.01263	eval-logloss:0.02586
[4360]	train-logloss:0.01257	eval-logloss:0.02581
[4370]	train-logloss:0.01252	eval-logloss:0.02577
[4380]	train-logloss:0.01246	eval-logloss:0.02574
[4390]	train-logloss:0.01240	eval-logloss:0.02570
[4400]	train-logloss:0.01236	eval-logloss:0.02567
[4410]	train-logloss:0.01231	eval-logloss:0.02563
[4420]	train-logloss:0.01226	eval-logloss:0.02559
[4430]	train-logloss:0.01220	eval-logloss:0.02555
[4440]	train-logloss:0.01216	eval-logloss:0.02552
[4450]	train-logloss:0.01211	eval-logloss:0.02548
[4460]	train-logloss:0.01207	eval-logloss:0.02545
[4470]	train-logloss:0.01203	eval-logloss:0.02542
[4480]	train-logloss:0.01200	eval-logloss:0.02539
[4490]	train-logloss:0.01196	eval-logloss:0.02537
[4500]	train-logloss:0.01192	eval-logloss:0.02534
[4510]	train-logloss:0.01188	eval-logloss:0.02532
[4520]	train-logloss:0.01184	eval-logloss:0.02529


[880]	train-logloss:0.05709	eval-logloss:0.06145
[890]	train-logloss:0.05666	eval-logloss:0.06103
[900]	train-logloss:0.05624	eval-logloss:0.06063
[910]	train-logloss:0.05593	eval-logloss:0.06033
[920]	train-logloss:0.05556	eval-logloss:0.06000
[930]	train-logloss:0.05522	eval-logloss:0.05967
[940]	train-logloss:0.05493	eval-logloss:0.05939
[950]	train-logloss:0.05457	eval-logloss:0.05907
[960]	train-logloss:0.05425	eval-logloss:0.05879
[970]	train-logloss:0.05396	eval-logloss:0.05851
[980]	train-logloss:0.05356	eval-logloss:0.05815
[990]	train-logloss:0.05319	eval-logloss:0.05781
[1000]	train-logloss:0.05280	eval-logloss:0.05746
[1010]	train-logloss:0.05247	eval-logloss:0.05715
[1020]	train-logloss:0.05212	eval-logloss:0.05683
[1030]	train-logloss:0.05186	eval-logloss:0.05660
[1040]	train-logloss:0.05153	eval-logloss:0.05630
[1050]	train-logloss:0.05118	eval-logloss:0.05599
[1060]	train-logloss:0.05090	eval-logloss:0.05573
[1070]	train-logloss:0.05062	eval-logloss:0.05547
[1080]	train

[2530]	train-logloss:0.02641	eval-logloss:0.03504
[2540]	train-logloss:0.02632	eval-logloss:0.03497
[2550]	train-logloss:0.02623	eval-logloss:0.03489
[2560]	train-logloss:0.02611	eval-logloss:0.03480
[2570]	train-logloss:0.02597	eval-logloss:0.03468
[2580]	train-logloss:0.02587	eval-logloss:0.03461
[2590]	train-logloss:0.02578	eval-logloss:0.03453
[2600]	train-logloss:0.02567	eval-logloss:0.03446
[2610]	train-logloss:0.02559	eval-logloss:0.03440
[2620]	train-logloss:0.02548	eval-logloss:0.03433
[2630]	train-logloss:0.02538	eval-logloss:0.03424
[2640]	train-logloss:0.02528	eval-logloss:0.03416
[2650]	train-logloss:0.02518	eval-logloss:0.03408
[2660]	train-logloss:0.02507	eval-logloss:0.03400
[2670]	train-logloss:0.02499	eval-logloss:0.03394
[2680]	train-logloss:0.02489	eval-logloss:0.03386
[2690]	train-logloss:0.02481	eval-logloss:0.03380
[2700]	train-logloss:0.02472	eval-logloss:0.03373
[2710]	train-logloss:0.02460	eval-logloss:0.03363
[2720]	train-logloss:0.02451	eval-logloss:0.03356


[4170]	train-logloss:0.01388	eval-logloss:0.02576
[4180]	train-logloss:0.01384	eval-logloss:0.02573
[4190]	train-logloss:0.01379	eval-logloss:0.02569
[4200]	train-logloss:0.01374	eval-logloss:0.02564
[4210]	train-logloss:0.01369	eval-logloss:0.02561
[4220]	train-logloss:0.01364	eval-logloss:0.02557
[4230]	train-logloss:0.01359	eval-logloss:0.02554
[4240]	train-logloss:0.01354	eval-logloss:0.02550
[4250]	train-logloss:0.01348	eval-logloss:0.02545
[4260]	train-logloss:0.01344	eval-logloss:0.02543
[4270]	train-logloss:0.01337	eval-logloss:0.02538
[4280]	train-logloss:0.01328	eval-logloss:0.02532
[4290]	train-logloss:0.01322	eval-logloss:0.02529
[4300]	train-logloss:0.01317	eval-logloss:0.02525
[4310]	train-logloss:0.01311	eval-logloss:0.02521
[4320]	train-logloss:0.01306	eval-logloss:0.02517
[4330]	train-logloss:0.01301	eval-logloss:0.02514
[4340]	train-logloss:0.01298	eval-logloss:0.02512
[4350]	train-logloss:0.01294	eval-logloss:0.02509
[4360]	train-logloss:0.01287	eval-logloss:0.02503


In [31]:
pred1 = models[0].predict(xgb.DMatrix(x_test))

In [32]:
pred2 = models[1].predict(xgb.DMatrix(x_test)) 

In [33]:
pred3 = models[2].predict(xgb.DMatrix(x_test))

In [34]:
pred4 = models[3].predict(xgb.DMatrix(x_test))

In [35]:
pred5 = models[4].predict(xgb.DMatrix(x_test)) 

In [36]:
pred_avg = (pred1 + pred2 + pred3 + pred4 + pred5)/5.0 

In [37]:
pred_avg

array([3.3375961e-03, 8.4028634e-06, 1.9242230e-03, ..., 9.9998724e-01,
       9.9995530e-01, 9.9995530e-01], dtype=float32)

In [38]:
np.save('xgb_p1.npy',pred1) 
np.save('xgb_p2.npy',pred2)
np.save('xgb_p3.npy',pred3) 
np.save('xgb_p4.npy',pred4)
np.save('xgb_p5.npy',pred5)

In [39]:
class_pred = np.where(pred_avg > 0.5, 1, 0).reshape(-1)


In [40]:
class_pred

array([0, 0, 0, ..., 1, 1, 1])